# USO LLMS PARA ANALIZAR TEXTOS


## Ejemplos de uso
- Obtención términos relevantes en el texto
- Resumen del texto

In [0]:
import os
import pandas as pd

from loguru import logger
from mlflow.deployments import get_deploy_client

from requests.exceptions import HTTPError

import re
import ast

[Monitor] stop monitoring


In [0]:
path = "/dbfs/FileStore/70822317g@santalucia.es/real_madrid_gana_liga.csv"
spanish_league_text = pd.read_csv(path)["texto"][0]

In [0]:
def _call_llm_model(prompt: str) -> str:
    """
    Función que llama al modelo LLM para hacer una predicción a partir de un prompt

    Args:
        prompt (str): Mensaje del prompt

    Returns:
        response (str): Respuesta del modelo si no hay fallo en la conexión.
        En otro caso, devuelve el mensaje de error
    """
    try:
        client = get_deploy_client("databricks")
        response = client.predict(
            endpoint="paa-gpt-4o-v20240513-dedicado-analitica",
            inputs={"messages": [{"role": "user", "content": prompt}]}
        )
        response = response["choices"][0]["message"]["content"]
        logger.info("Ejecución del LLM realizada")
    except HTTPError as e:
        if e.response.text:
            logger.info(f"{e}. Response text: {e.response.text}")
        else:
            logger.info(e)
        response = e
    return response

In [0]:
def _add_business_question_prompt(prompt: str, business_question: str) -> str:
  """
  Función que añade al prompt la cuestión de negocio

    Args:
        prompt (str): Mensaje del prompt
        business_question (str): Pregunta de negocio a clasificar

    Returns:
        response (str): Respuesta del modelo si no hay fallo en la conexión.
        En otro caso, devuelve el mensaje de error
  """
  prompt_end = " ".join(prompt)
  prompt_end = prompt_end.replace("{business_question}", business_question)
  return prompt_end

def _extraer_lista(text: str) -> list:
  """
  Función que extrae la lista del texto respuesta proporcionado por el LLM

  Args:
      text (str): Texto respuesta del LLM

  Returns:
      response (str): Respuesta del modelo si no hay fallo en la conexión.
      En otro caso, devuelve el mensaje de error
  """
  # Usar expresiones regulares para encontrar el contenido entre corchetes
  patron = re.search(r'\[.*\]', text, re.DOTALL)
  
  if patron:
    # Convertir el contenido de la lista de cadena a lista de Python
    output = ast.literal_eval(patron.group(0))
  else:
    output = []
  return output

### Ejemplo 1

Definimos el prompt con las instrucciones adecuadas:
- Obtener una lista de términos relevantes del texto
- Incluye ejemplo para ayudar a que el LLM aprenda a realizar la operación

In [0]:
prompt_1 = [
  "SYSTEM: Eres un asistente que analiza un texto y extrae los términos más relevantes y que definan el contenido del texto\n\n",
  "USER: A partir del siguiente texto: '{business_question}'\n\n",
  "extra en una lista los 10 términos más importanes que caractericen el texto. Que la lista sea en formato python"
  "INSTRUCTIONS: \n\n",
  "1. Presta atención al texto y devuelve los términos en formato lista de términos: [term1, term2, term3]\n\n"
  "Ejemplo de uso\n\n"
  "Pregunta: Soy Pablo Sánchez Cabrera, profesor del máster de la Uned de Big Data Science y data scientist en Santalucía. Me gusta el fútbol y la literatura. Quiero mostrar a los alumnos del curso un ejemplo típico de uso de LLMs\n\n",
  "Respuesta: [Pablo Sánchez Cabrera, profesor, Uned, Big Data Science, Santalucía, futbol, literatura, alumnos, LLMs, ejemplo]"
]

In [0]:
prompt_end = _add_business_question_prompt(prompt_1, spanish_league_text)
list_of_terms = _call_llm_model(prompt_end)
_extraer_lista(list_of_terms)

2024-06-28 20:24:23.168 | INFO     | __main__:_call_llm_model:19 - Ejecución del LLM realizada


['Real Madrid',
 'Liga',
 'Carlo Ancelotti',
 'Barcelona',
 'campeones',
 'Tebas',
 'negligencia arbitral',
 'lesiones',
 'temporada 2023-2024',
 'Jude Bellingham']

### Ejemplo 2


Definimos el prompt con las instrucciones adecuadas:
- Resume el texto que se proporciona

In [0]:
prompt_2 = [
  "SYSTEM: Eres un asistente que analiza un texto y hace un resumen del mismo\n\n",
  "USER: A partir del siguiente texto: '{business_question}' resume dicho texto entre 50-100 palabras"
  "INSTRUCTIONS: \n\n",
  "Presta atención al texto y haz un breve resumen. No incluyas más de 100 palabras\n\n"
]

In [0]:
prompt_end = _add_business_question_prompt(prompt_2, spanish_league_text)
_call_llm_model(prompt_end)

2024-06-28 20:24:25.269 | INFO     | __main__:_call_llm_model:19 - Ejecución del LLM realizada


'El Real Madrid ha ganado su 36ª Liga tras vencer al Cádiz y aprovechando el empate del Barcelona. El autor celebra este logro, reservando críticas para los árbitros, la Federación Española de Fútbol, y acusando al Barcelona de corrupción en años anteriores. Recalca las dificultades superadas por el equipo, como lesiones importantes y decisiones arbitrales controversiales, y elogia el trabajo de Carlo Ancelotti y el rendimiento de jugadores como Jude Bellingham y Lucas Vázquez. Destaca que, a pesar de las adversidades, el Real Madrid ha sido implacable y dominante en la competición, mereciendo plenamente el título.'